In [ ]:
%maven org.dflib:dflib-jupyter:2.0.0-SNAPSHOT

## Streaks

In [ ]:
var games = Parquet.loader()
    .cols("abbrev", "game_date", "game_id", "decision")
    .load("_data/team_game.parquet");

games

In [ ]:
var streaks1 = games
    .cols("streak_points", "up_or_down")
        .merge(
            ifExp($str("decision").eq("W"), $val(2), ifExp($str("decision").eq("O"), $val(1), $val(0))),
            ifExp($str("decision").in("W", "O"), $val(true), $val(false))
        )
    // sorting is super important for streaks
    .sort($col("abbrev").asc(), $col("game_date").asc());

streaks1

In [ ]:
var streaks2 = streaks1
    .cols("streak_id").merge($bool("up_or_down").ne( $bool("up_or_down").shift(1)).cumSum())
    .colsExcept("up_or_down").select()
    .group("abbrev", "streak_id").cols("streak").merge($int("streak_points").cumSum());

streaks2

In [ ]:
var streaks3 = streaks2
    .pivot().cols("abbrev").rows("game_date").vals("streak")
    .sort($col("game_date").asc());

streaks3

In [ ]:
var timeSpan = streaks3
    .cols("start_date", "end_date").agg($date("game_date").min(), $date("game_date").max())
    .cols("days").merge($date(0).mapVal($date(1), (d1, d2) -> java.time.temporal.ChronoUnit.DAYS.between(d1, d2)));

LocalDate from = (LocalDate) timeSpan.get(0, 0);
Series<LocalDate> daysSeq = new org.dflib.series.IntSequenceSeries(0, ((Long) timeSpan.get(2, 0)).intValue())
    .map(from::plusDays);

var date = DataFrame.byColumn("date").of(daysSeq);
date

In [ ]:
var streaks4 = date
    .leftJoin(streaks3).on("date", "game_date").colsExcept("game_date").select()
    .cols().fillNullsForward();

streaks4

In [ ]:
Series<String> teams = streaks4.getColumnsIndex().toSeries().head(-1);
Series<String> unselectedTeams = teams.select(not($col(0).in("NYR", "PIT")));

var p1 = ECharts.chart()
    .legend(Legend.ofPlain().vertical().leftRight().topPct(10).unselected(unselectedTeams.toArray(new String[0])))
    .grid(Grid.of().leftLeft().rightPct(25))
    .renderAsSvg()
    .tooltip(Tooltip.ofAxis())
    .toolbox(Toolbox.of().featureSaveAsImage().featureDataZoom().featureRestore())
    
    .sizePx(1000, 400)
    .xAxis("date", XAxis.ofTime())
    .series(SeriesOpts.ofLine().areaStyle().showSymbol(false), teams.toArray(new String[0]))
    .plot(streaks4);

p1

In [ ]:
var outcomes = games.group("abbrev", "decision")
    .agg(
        $col("abbrev"),
        $col("decision"),
        count());

outcomes

In [ ]:
var splitOutcomes = outcomes
    .cols("points").merge(ifExp($col("decision").eq("W"), $val(2), ifExp($col("decision").eq("O"), $val(1), $val(0))).castAsInt().mul($int("count")))
    .sort($int("points").desc())
    .pivot().cols("abbrev").rows("decision").vals("count")
    .cols("order").merge(ifExp($col("decision").eq("W"), $val(0), ifExp($col("decision").eq("O"), $val(1), $val(2))))
    .sort($col("order").asc())

;
splitOutcomes

In [ ]:
var e2 = ECharts.chart()
    .renderAsSvg()
    .tooltip(Tooltip.ofAxis())
    .toolbox(Toolbox.of().featureSaveAsImage().featureDataZoom().featureRestore())
    .legend(Legend.ofPlain().unselected("O"))
    .sizePx(1000, 400);

Series<String> teamsX = splitOutcomes.getColumnsIndex().toSeries().head(-1).tail(-1);

int len = teamsX.size();
int rowLen = 10;
int rows = len / rowLen + (len % rowLen > 0 ? 1 : 0);
for(int i = 0; i < len; i++) {
    String t = teamsX.get(i);
    double h = 8 + 8 * (i % rowLen);
    double v =  18 + 20 * (i / rowLen);
    e2.series(SeriesOpts.ofPie()
              .label("decision", PieLabel.ofCenter().formatter("{a}"))
              .itemStyle(PieItemStyle.of().borderType(LineType.solid).borderWidth(1).borderColor("black"))
              .radiusPct(16, 9)
              .centerPct(h, v)
              .name(t), t);
}

var p2 = e2.plot(splitOutcomes);
p2

In [ ]:
var goals = Parquet.loader()
    .cols("game_id", "scoring_team.abbrev", "scored_against_team.abbrev", "seconds_elapsed_in_game", "period")
    .load("_data/goals.parquet")
    .cols().as(s -> s.replace("team.abbrev", "team"));
    
goals

In [ ]:
var firstScoredForByGame = goals
    .colsExcept("scored_against_team", "period").select()
    .cols("game_mins").merge($int("seconds_elapsed_in_game").div(60.0))
    .sort($col("game_id").asc(), $col("game_mins").asc())
    .group("game_id").head(1).cols("scoring_team", "game_mins").select();

firstScoredForByGame

In [ ]:
var firstScoredFor = firstScoredForByGame.group("scoring_team")
    .cols("team", "min", "q1", "median", "q3", "max")
    .agg(
        $col("scoring_team"),
        $double("game_mins").min().mapVal(n -> Math.round((double) n)),
        $double("game_mins").quantile(0.25).mapVal(n -> Math.round((double) n)),
        $double("game_mins").quantile(0.5).mapVal(n -> Math.round((double) n)),
        $double("game_mins").quantile(0.75).mapVal(n -> Math.round((double) n)),
        $double("game_mins").max().mapVal(n -> Math.round((double) n))
    )
    .sort($col("median").asc());

firstScoredFor

In [ ]:
var p3 = ECharts.chart()
    .renderAsSvg()
    .toolbox(Toolbox.of().featureSaveAsImage().featureDataZoom().featureRestore())
    .tooltip(Tooltip.ofAxis())
    .sizePx(1000, 400)
    .xAxis("team", XAxis.ofCategory().label(
        AxisLabel.of().rotate(90).fontSize(9)
    ))
    .series(SeriesOpts.ofBoxplot().itemStyle(BoxplotItemStyle.of().color("#FF7043").borderColor("#000").borderWidth(1)),  "min", "q1", "median", "q3", "max")
    .plot(firstScoredFor);

p3

In [ ]:
ECharts.saver()
    .htmlTemplate("hockey_teams.mustache")
    .save("../_charts/hockey_teams.html", p1, p2, p3)